In [1]:
import random
import numpy as np
import torch
import torch.utils.data
import glob
import layers
from utils import load_wav_to_torch, load_filepaths_and_text
from text import text_to_sequence
import hparams
import os
from pathlib import Path
hparams = hparams.create_hparams()
import warnings
warnings.filterwarnings('ignore')


/home/xliu0/anaconda3/envs/taco/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xliu0/anaconda3/envs/taco/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xliu0/anaconda3/envs/taco/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xliu0/anaconda3/envs/taco/lib/python3.6/site-pack

In [2]:
stft = layers.TacotronSTFT(
            hparams.filter_length, hparams.hop_length, hparams.win_length,
            hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
            hparams.mel_fmax)

def get_mel(filename):
    audio, sampling_rate = load_wav_to_torch(filename)
    if sampling_rate != stft.sampling_rate:
        raise ValueError("{} {} SR doesn't match target {} SR".format(
            sampling_rate, stft.sampling_rate))
    audio_norm = audio / hparams.max_wav_value
    audio_norm = audio_norm.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = torch.squeeze(melspec, 0)
    melspec = melspec.data.numpy()
    return melspec

def save_spec(data, filepath, output_filename):
    if not(os.path.exists(filepath)):
        Path(filepath).mkdir(parents=True)
    np.save(os.path.join(filepath, output_filename), data)


## IEMOCAP


In [ ]:
SESS_PATH = '/data3/IEMOCAP/IEMOCAP_full_release/Session1/sentences/wav/'
for sess in list(range(1,6)):
    temp_sess_path = SESS_PATH.replace('1', str(sess))
    all_wav_path = sorted(glob.glob(temp_sess_path + '*/*.wav'))
    for wav_path in all_wav_path:
        spec = get_mel(wav_path)
        output_filename = wav_path.split('/')[-1].split('.')[0]
        spec_path = Path(wav_path.replace('IEMOCAP_full_release', 'IEMOCAP_full_release_spec'))
        spec_path = str(spec_path.parent)
        save_spec(spec, spec_path, output_filename)
    print('Finished Session ', str(sess))
        

## VCTK

In [ ]:
VCTK_PATH = '/data3/VCTK/VCTK-Corpus/wav48/'
for wav_path in sorted(glob.glob(VCTK_PATH + '*/*.wav')):
    spec = get_mel(wav_path)
    output_filename = wav_path.split('/')[-1].split('.')[0]
    spec_path = Path(wav_path.replace('/VCTK/', '/VCTK_spec/'))
    spec_path = str(spec_path.parent)
    save_spec(spec, spec_path, output_filename)    

# LJ

In [ ]:
LJ_PATH = '/data3/LJSpeech-1.1/wavs/'
for wav_path in sorted(glob.glob(LJ_PATH + '*.wav')):
    spec = get_mel(wav_path)
    output_filename = wav_path.split('/')[-1].split('.')[0]
    spec_path = Path(wav_path.replace('/LJSpeech-1.1/', '/LJSpeech-1.1-spec/'))
    spec_path = str(spec_path.parent)
    save_spec(spec, spec_path, output_filename)
